### Read the results csv

Analyse what we got from the evaluation of the locators

In [1]:
import pandas as pd
import os
import numpy as np


def extract_files_for_project(project_name, target_dir):
    lg_file,num_located,skipped,locators=None,None,None,None
    for file_name in os.listdir(target_dir):
        if file_name.startswith("locators_generation") and file_name.endswith(f"{project_name}.csv"):
            lg_file=file_name
        elif file_name.endswith(f"{project_name}_located.csv"):
            num_located=file_name
        elif file_name.endswith(f"{project_name}_skipped.csv"):
            skipped=file_name
        elif file_name.endswith(f"{project_name}.csv"):
            locators=file_name
    return lg_file,num_located,skipped,locators

root_dir="C:\\Users\\hilal.taha\\PycharmProjects\\pythonProject\\database\\statistics\\"
lg_files,num_locateds,skippeds,locatorss=[],[],[],[]
for model in os.listdir(root_dir+"trained_models"):
    project_name = model[:len(model) - 14]
    lg_file, num_located, skipped, locators = extract_files_for_project(project_name, root_dir+"evaluation_data\\")
    if lg_file is None or num_located is None or skipped is None or locators is None:
        continue
    lg_files.append(lg_file)
    num_locateds.append(num_located)
    skippeds.append(skipped)
    locatorss.append(locators)

## Calculate the accuracies and robustness

We loop per row, calculate the accuracy(recoil) on each row, seperating the SEL and RELOC locators
Then at the end of the rows, we divide on the number of rows to get the average accuracy.

### Robustness 
It is the duration of survival of the locator

In [3]:
import re

def calculate_duration_of_survival(col,dos=1,):
    survival_duration = 0
    consecutive_zeros = 0
    for value in col[::-1]:
        if value == 0:
            consecutive_zeros += 1
            if consecutive_zeros == dos:
                break
        else:
            survival_duration += 1
            consecutive_zeros = 0
    return survival_duration

def calculate_locator_length(col):
    locator_name = col.name  # Get the column name (locator name)
    locator_parts = re.split('/', locator_name.replace('//','/'))  # Split on '/ ' and '//'
    locator_length = len(locator_parts)
    if locator_name[-6:]=="_RELOC" or locator_name[:5]=="xpath":
        locator_length=locator_length-1
    return locator_length

def calculate_accuracy(col):
    TP=0
    FP=0
    for value in col[::-1]:
        if value > 0:
            TP+=1
            FP+=value-1
    if TP+FP==0:
        return 0
    return TP/(TP+FP)

def replace_greater_than_one(x):
    return 1 if x > 1 else x

def compute_reloc_recoil(row):
    row_series = pd.Series(row)
    positives_count = (row_series[reloc_columns] > 0).sum()
    zero_count = len(row_series[reloc_columns]) - positives_count
    tp_count = positives_count
    fp_count =  row_series[reloc_columns].sum() - positives_count
    if tp_count + fp_count == 0:
        selected_ratio = 0  # Avoid division by zero
    else:
        selected_ratio = tp_count / (tp_count + fp_count)
    
    return selected_ratio


def compute_sel_recoil(row):
    row_series = pd.Series(row)
    
    positives_count = (row_series[sel_columns] > 0).sum()
    zero_count = len(row_series[sel_columns]) - positives_count
    tp_count = positives_count
    fp_count =  row_series[sel_columns].sum() - positives_count
    
    
    if tp_count + fp_count == 0:
        selected_ratio = 0  # Avoid division by zero
    else:
        selected_ratio = tp_count / (tp_count + fp_count)
    
    return selected_ratio

reloc_columns = None
sel_columns=[]
xpath_sel_columns = None
class_sel_columns = None
name_sel_columns = None
id_sel_columns = None
tag_sel_columns = None
link_sel_columns = None
all_columns = None
accuracies_RELOC=[]
accuracies_SEL=[]
dos_sel=[]
dos_RELOC=[]
df_d=None

for file in num_locateds:
    # Read the CSV file
#     print(file)
    try:
        df = pd.read_csv(root_dir+"evaluation_data\\"+file, index_col="Timestamp")
    except Exception as e:
        print(e)
    # Step 2: Calculate the duration of survival for each locator
    reloc_columns = df.columns[df.columns.str.endswith("_RELOC")]
#     sel_columns = df.columns[df.columns.str.endswith("_SEL")]
    xpath_sel_columns = df.columns[df.columns.str.startswith("xpath") & df.columns.str.endswith("_SEL")]
    class_sel_columns = df.columns[df.columns.str.endswith("_SEL") & df.columns.str.startswith("class name")]
    name_sel_columns = df.columns[df.columns.str.endswith("_SEL") & df.columns.str.startswith("name")]
    id_sel_columns = df.columns[df.columns.str.endswith("_SEL") & df.columns.str.startswith("id")]
    tag_sel_columns = df.columns[df.columns.str.endswith("_SEL") & df.columns.str.startswith("tag name")]
    link_sel_columns = df.columns[df.columns.str.endswith("_SEL") & df.columns.str.startswith("link text")]
    all_columns = df.columns[df.columns.str.endswith("_RELOC") | df.columns.str.endswith("_SEL")]
#     df[xpath_sel_columns] = df[xpath_sel_columns].applymap(replace_greater_than_one)

    max_dos=-1
    column_to_add=None
    sel_columns=[]
    for col in df.columns:
        if col.endswith("_SEL"):
            dos= calculate_duration_of_survival(col)
            if dos>max_dos:
                max_dos=dos
                column_to_add=col
            if col.startswith("xpath"):
                sel_columns.append(column_to_add)
                max_dos=-1
                column_to_add=None
    sel_columns=pd.Index(sel_columns)
    
    # Iterate over all rows in the DataFrame using iterrows()
    total_acc_RELOC = 0
    total_acc_SEL = 0
    num_rows = len(df)

    for index, row in df.iterrows():
        total_acc_RELOC += compute_reloc_recoil(row)
        total_acc_SEL += compute_sel_recoil(row)
    average_acc_RELOC = total_acc_RELOC / num_rows
    average_acc_SEL = total_acc_SEL / num_rows
    
#     accuracy = df[all_columns].apply(calculate_accuracy)
#     accuracy = df[xpath_sel_columns].apply(calculate_accuracy)

    duration_of_survival = df[all_columns].apply(calculate_duration_of_survival)
    total_dos=0
    for index, dos in enumerate(duration_of_survival[sel_columns].values):
        total_dos=total_dos+dos
    total_dos= total_dos/len(duration_of_survival)
    dos_sel.append(total_dos)
    
    total_dos=0
    for index, dos in enumerate(duration_of_survival[reloc_columns].values):
        total_dos=total_dos+dos
    total_dos= total_dos/len(duration_of_survival)
    dos_RELOC.append(total_dos)
#     duration_of_survival = df[xpath_sel_columns].apply(calculate_duration_of_survival)
    
    locator_length = df[all_columns].apply(calculate_locator_length)
#     correlation_accuracy = accuracy.corr(duration_of_survival)
#     correlation_accuracy = accuracy.corr(locator_length)
    correlation_length = locator_length.corr(duration_of_survival)
#     print("Accuracy :"+str(correlation_accuracy))
#     print("Length :"+str(correlation_length))
    if average_acc_RELOC>1:
        print(file)
        df_d = df
    accuracies_RELOC.append(average_acc_RELOC)
    accuracies_SEL.append(average_acc_SEL)



unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
unsupported operand type(s) for -: 'str' and 'int'
[0.9838709677419355, 0.952755905511811, 1.0, 0.0, 0.975, 0.9890710382513661, 0.9543147208121827, 0.9797979797979798, 0.9425287356321839, 0.9041796528948216, 1.0, 1.0, 0.9948979591836735, 0.0, 0.995, 0.7738693467336684, 1.0, 0.9646153846153845, 0.9424083769633508, 0.34615384615384615, 0.9650822510822514, 1.0, 0.8943384757025996, 0.12610858529225877, 0.8059049079754599, 0.075, 0.9181818181818184, 1.0, 0.9949494949494949, 0.85, 0.93, 1.0, 1.0, 1.0, 0.12610858529225877, 0.9973333333333333,

In [9]:
from scipy import stats


alpha = 0.05
# Perform the Wilcoxon signed-rank test
statistic, p_value = stats.wilcoxon(dos_RELOC, accuracies_RELOC)

print("Wilcoxon statistic RELOC:", statistic)
print("p-value RELOC:", p_value)

if p_value < alpha:
    print("Reject the null hypothesis: The distributions are different.")
else:
    print("Fail to reject the null hypothesis: The distributions are similar.")
print()
statistic, p_value = stats.wilcoxon(dos_sel, accuracies_SEL)

print("Wilcoxon statistic SEL:", statistic)
print("p-value SEL:", p_value)

if p_value < alpha:
    print("Reject the null hypothesis: The distributions are different.")
else:
    print("Fail to reject the null hypothesis: The distributions are similar.")


Wilcoxon statistic RELOC: 618.0
p-value RELOC: 1.2918945362457503e-14
Reject the null hypothesis: The distributions are different.

Wilcoxon statistic SEL: 534.0
p-value SEL: 1.3138032799678164e-15
Reject the null hypothesis: The distributions are different.
